In [ ]:
# Import packages
from collections import Counter
import csv
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
import statistics as st
from tcn import TCN
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

Data preparation - Binning dwell times

In [ ]:
# Import data using pandas
data = pd.read_csv("browsing_train.csv") 
# View first rows of data
data.head()

In [ ]:
# Calculate dwell times for every row
data['dwell_times'] = abs(data['server_timestamp_epoch_ms'].diff(-1))
# View first rows of data
data.head()

In [ ]:
# Calculate amount of zero millisecond dwell times in the total dataset
zeros = data['dwell_times'].value_counts()[0.0]
rows = data.shape[0]
percentageofzeros = round(((zeros/rows)*100),2)

print("Number of zero dwell times: {}, number of dwell times: {}, percentage of zero milliseconds dwell times: {}".format(zeros,rows,percentageofzeros))

In [ ]:
# Save this dataset into csv file
df = pd.DataFrame(data) 
df.to_csv('browsing_train_dwelltimes.csv', index=False) 

In [ ]:
user_sessions = []
current_session_id = None
current_session = []

# Read dataset
with open("browsing_train_dwelltimes.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for idx, row in enumerate(reader):
            
        # Row will contain: session_id_hash, product_action, product_sku_hash
        _session_id_hash = row['session_id_hash']
        # When a new session begins, store the old one and start again
        if current_session_id and current_session and _session_id_hash != current_session_id:
            user_sessions.append(current_session)
            # Resets session
            current_session = []
        # We extract events from session
        if row['product_action'] == '' and row['event_type'] ==  'pageview':
            current_session.append('view')

        elif row['product_action'] != '':
            current_session.append(row['product_action'])
        
        # Append dwell times between actions
        if row['dwell_times'] == '':
            current_session.append("nan")
        else:
            current_session.append(float(row['dwell_times']))
            
        # Update the current session id
        current_session_id = _session_id_hash
        

# Print how many sessions we have
print("# total sessions: {}".format(len(user_sessions)))
# Print first session
print("First session is: {}".format(user_sessions[0]))

In [ ]:
# If the last value of a user session is of float value, delete this value
# As this value is the time between sessions
for sublist in user_sessions:
    if type(sublist[-1]) == float or (sublist[-1]) == "nan":
        del(sublist[-1])
    else:
        continue
assert not any( x[-1] == float and x[-1] == "nan" for x in user_sessions)

In [ ]:
# Print first session again
print(user_sessions[0])

In [ ]:
# The calculated time between sessions are very large
# Before we continue, we need to figure out the largest dwell time that is actually
# inter-click
floatvalues = []
for session in user_sessions:
    for item in session:
        if type(item) == float:
            floatvalues.append(item)
        else:
            continue
largestdwelltime = max(floatvalues)
print("Largest inter-click dwell time:", largestdwelltime)

In [ ]:
# Import original data again
data = pd.read_csv("browsing_train.csv") 
# Calculate dwell times for every row
data['dwell_times'] = abs(data['server_timestamp_epoch_ms'].diff(-1))
# We change the time between sessions to the largest inter-click dwell time 
# This is necessary for the binning of dwell times later 
data.loc[data['dwell_times'] > largestdwelltime, 'dwell_times'] = largestdwelltime

In [ ]:
if data['dwell_times'].max() == largestdwelltime:
    print("Ok!")

In [ ]:
# Bin dwell times into quartiles
data["dwell_bins"] = pd.qcut(
    x=data["dwell_times"],
    q = 5,
    labels=["Very low", "Low", "Medium", "High"],
    duplicates = "drop"
)

In [ ]:
# View first rows of data
data.head()

In [ ]:
# Print edge values of bins
print(data['dwell_bins'].unique())

In [ ]:
# Print labels of bins
print(data['dwell_bins'].unique())

In [ ]:
# Save this dataset into csv file again
df = pd.DataFrame(data) 
df.to_csv('browsing_train_dwelltimes.csv', index=False) 

Data preparation

In [ ]:
user_sessions = []
current_session_id = None
current_session = []

with open("browsing_train_dwelltimes.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for idx, row in enumerate(reader):
            
        # Row will contain: session_id_hash, product_action, product_sku_hash
        _session_id_hash = row['session_id_hash']
        # When a new session begins, store the old one and start again
        if current_session_id and current_session and _session_id_hash != current_session_id:
            user_sessions.append(current_session)
            # Resets session
            current_session = []
        # We extract events from session
        if row['product_action'] == '' and row['event_type'] ==  'pageview':
            current_session.append('view')

        elif row['product_action'] != '':
            current_session.append(row['product_action'])
        
        # Append binned dwell times between actions
        if row['dwell_bins'] == '':
            current_session.append("nan")
        else:
            current_session.append(row['dwell_bins'])
            
        # update the current session id
        current_session_id = _session_id_hash
        

# Print how many sessions we have
print("# total sessions: {}".format(len(user_sessions)))
# Print first session
print("First session is: {}".format(user_sessions[0]))

In [ ]:
# If the last value of a user session is a binned dwell time, delete this value
# This is the time between sessions
labels=["Very low", "Low", "Medium", "High", "Very high"]
for sublist in user_sessions:
    if (sublist[-1]) in labels or (sublist[-1]) == "nan":
        del(sublist[-1])
    else:
        continue
assert not any( x[-1] in labels and x[-1] == "nan" for x in user_sessions)

In [ ]:
# Function to convert events to numbers and add start and stop token
def session_indexed(s):
    """
    Converts a session (of actions) to indices and adds start/end tokens
    :param s: list of actions in a session (i.e 'add','detail', etc)
    :return:
    """
    action_to_idx = {'start': 0, 'end': 1, 'add': 2, 'remove': 3, 'detail': 4, 'view': 5,
                    'Very low': 6, 'Low': 7, 'Medium': 8, 'High': 9}
    return [action_to_idx['start']] + [action_to_idx[e] for e in s] + [action_to_idx['end']]

In [ ]:
purchase_sessions = []
abandon_sessions = []
browse_sessions = []
for s in user_sessions:
    # If add and purchase event in sessions and purchase event appears after add event...
    if 'purchase' in s and 'add' in s and s.index('purchase') > s.index('add'):
        p_session = s
        # Remove purchase event and dwell time before purchase event
        p_session = (p_session[:p_session.index("purchase")][:-1])
            
        # Remove clickstreams shorter than 9 or longer than 309 clicks and dwell times
        if len(p_session) < 9 or len(p_session) > 309:
            continue
        else:
            # Append to list
            purchase_sessions.append(p_session)
        # Assert not any purchase event left in clickstream    
        assert not any( e == 'purchase' for e in p_session)

    # If add event and no purchase event in session...    
    elif 'add' in s and not 'purchase' in s:
        if len(s) < 9 or len(s) > 309:
            continue
        else:
            abandon_sessions.append(s)
            
    # If no purchase event in session...        
    elif 'purchase' not in s:
        if len(s) < 9 or len(s) > 309:
            continue
        else:    
            browse_sessions.append(s)

In [ ]:
# Add start stop token, convert to numbers
purchase_sessions = [session_indexed(s) for s in purchase_sessions]
abandon_sessions = [session_indexed(s) for s in abandon_sessions]
browse_sessions = [session_indexed(s) for s in browse_sessions]

# Combine sessions into final dataset
x = purchase_sessions + abandon_sessions + browse_sessions

# give label=1 for purchase, label=0 for abandon, label=2 for browse
y = [1]*len(purchase_sessions) +[0]*len(abandon_sessions) + [2]*len(browse_sessions)
assert len(x) == len(y)

Split data into train, val and test set

In [ ]:
from sklearn.model_selection import train_test_split
# First, split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(x,y, train_size=0.7, stratify = y, random_state = 3340)

In [ ]:
# Second, split the remaining data into a validation and test set
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5, stratify = y_rem, random_state = 3340)

In [ ]:
print("Size of training data: {}, validation data: {}, test data: {}".format(len(X_train), len(X_valid), len(X_test)))

Exploring data

In [ ]:
# Convert training set to seperate sets per class
def converttosessions(X_train, y_train):
    tupletrainitems = [tuple(x) for x in X_train]
    tuplex = tuple(tupletrainitems)
    tupley = tuple(y_train)
    newdic = zip(tupley,tuplex)
    
    abandon_sessions = []
    purchase_sessions = []
    browsing_sessions = []
    
    for x in list(newdic):
        if x[0] == 0:
            a = list(x[1])
            b = a[1:-1]
            abandon_sessions.append(b)
        elif x[0] == 1:
            a = list(x[1])
            b = a[1:-1]
            purchase_sessions.append(b)
        elif x[0] == 2:
            a = list(x[1])
            b = a[1:-1]
            browsing_sessions.append(b)
    
    return abandon_sessions, purchase_sessions, browsing_sessions

In [ ]:
train_abandon_sessions, train_purchase_sessions, train_browsing_sessions = converttosessions(X_train, y_train)

In [ ]:
# Count dwell times per class
def dwelltimesperclass(abandon_sessions, purchase_sessions, browsing_sessions):
    abandoncounter = Counter(itertools.chain(*abandon_sessions))
    abandoncounter = dict(sorted(abandoncounter.items(), key=lambda item: item[0]))
    purchasecounter = Counter(itertools.chain(*purchase_sessions))
    purchasecounter = dict(sorted(purchasecounter.items(), key=lambda item: item[0]))
    browsingcounter = Counter(itertools.chain(*browsing_sessions))
    browsingcounter = dict(sorted(browsingcounter.items(), key=lambda item: item[0]))
    
    abandon_list = list(abandoncounter.values())
    abandon_list = abandon_list[4:]
    purchase_list = list(purchasecounter.values())
    purchase_list = purchase_list[4:]
    browsing_list = list(browsingcounter.values())
    browsing_list = browsing_list[3:]
    

    print("Total number of events and dwell times per class:")
    print("Abandon: {}, purchase: {}, browsing: {}".format(abandoncounter, purchasecounter, browsingcounter))
    print("\n")
    print(abandon_list, purchase_list, browsing_list)
    total_dwell_abandon = sum(abandon_list)
    total_dwell_purchase = sum(purchase_list)
    total_dwell_browsing = sum(browsing_list)
    
    perc_verylowabandon = round(((abandon_list[0]/total_dwell_abandon) * 100),2)
    perc_verylowpurchase = round(((purchase_list[0]/total_dwell_purchase) * 100),2)
    perc_verylowbrowsing = round(((browsing_list[0]/total_dwell_browsing) * 100),2)    
    print("Percentage of very low dwell times in abandon: {}, purchase: {}, browsing: {}".format(perc_verylowabandon, perc_verylowpurchase, perc_verylowbrowsing))
    
    perc_lowabandon = round(((abandon_list[1]/total_dwell_abandon) * 100),2)
    perc_lowpurchase = round(((purchase_list[1]/total_dwell_purchase) * 100),2)
    perc_lowbrowsing = round(((browsing_list[1]/total_dwell_browsing) * 100),2)
    print("Percentage of low dell times in abandon: {}, purchase: {}, browsing: {}".format(perc_lowabandon, perc_lowpurchase, perc_lowbrowsing))
    
    perc_mediumabandon = round(((abandon_list[2]/total_dwell_abandon) * 100),2)
    perc_mediumpurchase = round(((purchase_list[2]/total_dwell_purchase) * 100),2)
    perc_mediumbrowsing = round(((browsing_list[2]/total_dwell_browsing) * 100),2)
    print("Percentage of medium dwell times in abandon: {}, purchase: {}, browsing: {}".format(perc_mediumabandon, perc_mediumpurchase, perc_mediumbrowsing))
    
    perc_highabandon = round(((abandon_list[3]/total_dwell_abandon) * 100),2)
    perc_highpurchase = round(((purchase_list[3]/total_dwell_purchase) * 100),2)
    perc_highbrowsing= round(((browsing_list[3]/total_dwell_browsing) * 100),2)
    print("Percentage of high dwell times in abandon: {}, purchase: {}, browsing: {}".format(perc_highabandon, perc_highpurchase, perc_highbrowsing))
    
    verylow_percentages = [perc_verylowabandon, perc_verylowpurchase, perc_verylowbrowsing]
    low_percentages = [perc_lowabandon, perc_lowpurchase, perc_lowbrowsing]
    medium_percentages = [perc_mediumabandon, perc_mediumpurchase, perc_mediumbrowsing]
    high_percentages = [perc_highabandon, perc_highpurchase, perc_highbrowsing]
    
    df = pd.DataFrame({'Class': ["Abandon", "Purchase", "Browsing-Only"], 'Very Low': verylow_percentages, 'Low': low_percentages,
                      'Medium': medium_percentages, 'High': high_percentages})
    print(df)
    df.plot(x='Class', kind='bar', stacked=True)
    plt.legend(bbox_to_anchor=(1.0, 1.0))
    plt.xticks(rotation=0)
    plt.ylabel("Percentage")
    plt.show()


In [ ]:
dwelltimesperclass(train_abandon_sessions, train_purchase_sessions, train_browsing_sessions)

In [ ]:
# Visualise clickstream
def visualizeclickstream(abandon_sessions, purchase_sessions, browsing_sessions):
    abandon_clickstream = 0
    purchase_clickstream = 0
    browsing_clickstream = 0
    length = list(range(1,39+1))
    
    for x in abandon_sessions:
        if len(x) == length[-1] and 3 in x:
            abandon_clickstream = x
    for x in purchase_sessions:
        if len(x) == length[-1] and 3 in x:
            purchase_clickstream = x
    for x in browsing_sessions:
        if len(x) == length[-1] and 3 in x:
            browsing_clickstream = x

    # Plot a simple line chart
    plt.figure(figsize=(8,4))

    # Plot another line on the same chart/graph
    plt.plot(length, purchase_clickstream, marker='p', color = 'tab:orange', linestyle='--')
    plt.axhline(y = 5.5, linestyle = '-', color = 'black')

    
    #{add': 2, 'remove': 3, 'detail': 4, 'view': 5, 'Very low': 6, 'Low': 7, 'Medium': 8, 'High': 9}
    y = [2,3,4,5,6,7,8,9]
    yticks = ['Add', 'Remove', 'Detail', 'View', "Very low", 'Low', 'Medium', 'High']
    xticks = list(range(0,41, 5))
    plt.yticks(y, yticks)
    plt.xticks(xticks)
    
    #Invert y-axis for legible plot
    plt.gca().invert_yaxis()
    plt.legend(['Purchase'])
    plt.xlabel("Number of Clicks and Inter-click Dwell Times")
    plt.ylabel("Dwell Time                 Event Type")
    plt.show()

In [ ]:
visualizeclickstream(train_abandon_sessions, train_purchase_sessions, train_browsing_sessions)

Padding & one-hot encoding

In [ ]:
# Pad sequences
max_len = 311
X_train = pad_sequences(X_train, padding="post",value=10, maxlen=max_len)
X_valid = pad_sequences(X_valid, padding="post", value=10, maxlen=max_len)
X_test = pad_sequences(X_test, padding="post", value=10, maxlen=max_len)

In [ ]:
# Convert to one-hot
X_train = tf.one_hot(X_train, depth=11)
X_valid = tf.one_hot(X_valid, depth=11)
X_test = tf.one_hot(X_test, depth=11)

In [ ]:
# Convert labels to arrays
y_train = np.array(y_train)
y_valid = np.array(y_valid)

#One-hot encode labels
y_train = to_categorical(y_train, 3)
y_valid = to_categorical(y_valid, 3)

# Save for metric calculations
test_labels = y_test

Gridsearch

CNN

In [ ]:
def cnngridsearch(filter_size, neurons, kernel_size):
    batch = 32
    epochs = 20
    patience = 5
    l = 0.001
    #Hyperparameters
    opt = keras.optimizers.Adam(l)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filter_size, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size = 2))    
    model.add(Flatten())
    model.add(Dense(neurons, activation='relu'))
    model.add(Dense(y_train.shape[1],activation='softmax'))
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
# Hyperparameters to be tested
neurons = [32, 64, 128]
kernel_num = [3, 5, 7]
filter_num = [32, 64, 128]
f1_scores = dict()

for n in neurons:
    print("Testing neurons:", n)
    for k in kernel_num:
        print("Testing kernel size:", k)
        for f in filter_num:
            print("Testing filters", f)
            print("Fitting model")
            cnn = cnngridsearch(f, n, k)
            
            #Calculating y_pred
            y_pred_validate = cnn.predict(X_valid)
            rounded = np.argmax(np.round(y_pred_validate),axis=1)
            rounded = list(rounded)
        
            #Evaluating the model
            f1score = f1_score(valid_labels, rounded, average = "macro")
            print("Macro-averaged F1 score:", f1score)
        
            #Appending evaluations to dictionaries
            f1_scores[f1score] = (f, n, k)

In [ ]:
#Sort F1 scores from highest to lowest
sortscores = {key: val for key, val in sorted(f1_scores.items(), key = lambda ele: ele[0])}
print("Result dictionary sorted by F1 score : " + str(sortscores))

In [ ]:
# Model using only one stack
def tcngridsearch(filter_size, kernel_size, dilations):
    epochs=20
    patience=5
    batch=32
    l=0.001
    
    #Hyperparameters
    opt = keras.optimizers.Adam(l)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)

    # Define Model
    model = keras.Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # TCN layer
    model.add(TCN(
        nb_filters= filter_size,
        kernel_size=kernel_size,
        dilations=dilations
        ))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))

    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
kernel_num = [3, 5, 7]
filter_num = [32, 64, 128]
dilations = [[1, 2, 4, 8, 16, 32, 64, 128], [1, 2, 4, 8, 16, 32, 64]]
f1_scores = dict()

# Calculate receptive field
print("Receptive fields")
print("For kernel size 3: {}".format(3*1*128))
print("For kernel size 5: {}".format(5*1*64))
print("For kernel size 7: {}".format(7*1*64))

In [ ]:
for f in filter_num:
    print("Testing filter size:", f)
    for k in kernel_num:
        print("Testing kernel size:", k)
        if k == 3:
            d = dilations[0]
            print("Last dilation:", d[-1])
        if k == 5 or k == 7:
            d = dilations[1]
            print("Last dilation:", d[-1])
        
        print("Fitting model")
        tcn = tcngridsearch(f, k, d)
            
        #Calculating y_pred
        y_pred_validate = tcn.predict(X_valid)
        rounded = np.argmax(np.round(y_pred_validate),axis=1)
        rounded = list(rounded)
        
        #Evaluating the model
        f1score = f1_score(valid_labels, rounded, average = "macro")
        print("f1 score for this model:", f1score)
        
        #Appending evaluations to dictionaries
        f1_scores[f1score] = (f, k, d)

In [ ]:
#Sort F1 scores from highest to lowest
sortscores = {key: val for key, val in sorted(f1_scores.items(), key = lambda ele: ele[0])}
print("Result dictionary sorted by F1 score : " + str(sortscores))

Testing models

In [ ]:
# Calculate F1 scores and configure confusion matrices
def metric_calculation(predictions):
    rounded = np.argmax(np.round(predictions),axis=1)
    rounded = list(rounded)
    f1 = f1_score(test_labels, rounded, average='macro')
    print ("f1macro: {}".format(round(f1, 3)))
    print(metrics.classification_report(test_labels, rounded, digits=3))
    
    target_names = ["Abandon", "Purchase", "Browsing-Only"]
    cm = metrics.confusion_matrix(test_labels, rounded)
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    heatmap = sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=target_names, yticklabels=target_names, cmap=plt.cm.Blues)
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    
    figure = heatmap.get_figure()    
    figure.savefig('cm.png')
    
    return heatmap

LSTM

In [ ]:
# Baseline
def lstmmodel(X_train, y_train, X_val, y_val):
    #Hyperparamaters
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    opt = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # LSTM layer
    model.add(LSTM(64)) 
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def lstmpredictions(X_test):
    lstm = lstmmodel(X_train, y_train, X_valid, y_valid)
    preds = lstm.predict(X_test,batch_size=32)
    return preds

In [ ]:
lstm_preds = lstmpredictions(X_test)

In [ ]:
metric_calculation(lstm_preds)

CNN

In [ ]:
#CNN baseline
def cnnmodel(X_train, y_train, X_val, y_val):
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    #Hyperparameters
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filters = 64, kernel_size = 7, activation='relu'))
    # Pooling layer
    model.add(MaxPooling1D(pool_size = 2))    
    # Flatten layer
    model.add(Flatten())
    # Fully connected layer
    model.add(Dense(32, activation='relu'))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def cnnpredictions(X_test):
    cnn = cnnmodel(X_train, y_train, X_valid, y_valid)
    preds = cnn.predict(X_test,batch_size=32)
    return preds

In [ ]:
cnn_preds = cnnpredictions(X_test)

In [ ]:
metric_calculation(cnn_preds)

CNN-LSTM

In [ ]:
def cnnlstmdwelltimes(X_train, y_train, X_val, y_val):
    lr = 0.001
    batch = 32
    epochs = 50
    patience = 10
    #Hyperparameters
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)
    
    model = Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # Convolutional layer
    model.add(Conv1D(filters = 64, kernel_size = 7, activation='relu'))
    # Pooling layer
    model.add(MaxPooling1D(pool_size = 2))  
    # LSTM layer
    model.add(LSTM(64))
    # Output layer
    model.add(Dense(y_train.shape[1],activation='softmax'))
    model.summary()
    
    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    
    return model

In [ ]:
def cnnlstmpredictions(X_test):
    cnnlstm = cnnlstmmodel(X_train, y_train, X_valid, y_valid)
    preds = cnnlstm.predict(X_test,batch_size=32)
    return preds

In [ ]:
cnnlstm_preds = cnnlstmpredictions(X_test)

In [ ]:
metric_calculation(cnnlstm_preds)

TCN

In [ ]:
def tcn_dwelltimes(X_train, y_train, X_val, y_val):
    epochs=50
    patience=10 
    batch=32
    lr=0.001
    
    opt = keras.optimizers.Adam(learning_rate = lr)
    loss = keras.losses.CategoricalCrossentropy()
    es = keras.callbacks.EarlyStopping(monitor='val_loss',
                                       patience=patience,
                                       verbose=1,
                                       restore_best_weights=True)

    # Define Model
    model = keras.Sequential()
    # Input layer
    model.add(Input(shape = (X_train.shape[1], X_train.shape[2])))
    # TCN layer
    model.add(TCN(
        nb_filters=32,
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128]
        ))
    # Output layer
    model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))
    model.summary()

    model.compile(optimizer=opt,
                loss=loss,
                metrics=['categorical_accuracy'])
    
    model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs = epochs,
                    batch_size = batch,
                    callbacks = es)
    return model

In [ ]:
def tcnpredictions(X_test):
    tcn = tcnmodel(X_train, y_train, X_valid, y_valid)
    preds = tcn.predict(X_test,batch_size=32)
    return preds

In [ ]:
tcn_preds = tcnpredictions(X_test)

In [ ]:
metric_calculation(tcn_preds)